In [62]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [45]:
def trend_break(df, stop_date):
    counties = df['Unnamed: 3']
    date_lst = [i for i in df.iloc[:,5:] if 'Unnamed' not in i]
    tstpos_lst = [i for i in df.iloc[:,5:] if df[i][0][0:6] == 'tstpos']
    mort_lst = [i for i in df.iloc[:,5:] if df[i][0][0:4] == 'mort']
    dic = {}
    for i in range(len(tstpos_lst)):
        if i == 0:
            continue
        date = date_lst[i]  

        if date == stop_date:
            break
        curr_c = df[tstpos_lst[i]]
        prev_c = df[tstpos_lst[i-1]]
        curr_c = curr_c[1:].fillna(0).replace(',','', regex=True).apply(int)
        prev_c = prev_c[1:].fillna(0).replace(',','', regex=True).apply(int)
        diff_c = curr_c - prev_c
        neg_idx_c = np.where(diff_c < 0)[0]
        # neg_idx_c = diff_c.index[diff_c < 0]
        tb_counties_c = counties[neg_idx_c]
        
        
        curr_d = df[mort_lst[i]]
        prev_d = df[mort_lst[i-1]]
        curr_d = curr_d[1:].fillna(0).replace(',','', regex=True).apply(int)
        prev_d = prev_d[1:].fillna(0).replace(',','', regex=True).apply(int)
        diff_d = curr_d - prev_d
        neg_idx_d = np.where(diff_d < 0)[0]
        # neg_idx_d = diff_d.index[diff_d < 0]
        tb_counties_d = counties[neg_idx_d]

        state_cases = tb_counties_c[tb_counties_c.str.contains(',')].str.split(',').apply(lambda x: x[-1])
        state_deaths = tb_counties_d[tb_counties_d.str.contains(',')].str.split(',').apply(lambda x: x[-1])

        # replace with tb_counties_c, tb_counties_d to get the specific counties
        exceptions = ['Georgia','Indiana','Michigan','Connecticut','North Dakota','Virginia','Massachusetts','Maine','Texas','Vermont',
            'Tennessee','Pennsylvania','California','Puerto Rico','Minnesota','Washington','Wisconsin','Illinois','Ohio']
        out_pre = pd.concat([state_cases, state_deaths]) 
        out = []
        for i in out_pre:
            if i.strip() not in exceptions:
                out.append(i.strip())
        if len(out) > 0:
            dic[date] = set(out)
     
    return dic

In [47]:
# insert all sheet urls here
url_lst = ['https://docs.google.com/spreadsheets/d/13gHl6iXtmV2yqTxS9Tn2aVe-L7dacnBA5BBn0jdhcas/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1Sp19_x01YL5usg_ghwUpC0GoFOlMKHRH671oDF3gp6M/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1KestNxi-AUi_mQu6CVaDHN8a5ud9qE3z4VzHGcHMX64/edit#gid=311031188',
'https://docs.google.com/spreadsheets/d/1xLIST_YPe9Amhc2c9ZLYV0gOiS3wtidtRM_KhyJMu_s/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1c34elyG4DcqZL9zcnGegrmIRD2OS_yl_LCsXQqCUY08/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1hvfXKUiZ0MW3IKup3WONi6gounAPzJLtMXG_5azhCC0/edit#gid=311031188',
'https://docs.google.com/spreadsheets/d/1N4o8nDhHVUUcylB4pHcWk9czKOLSEv-hjZcvExS3vSg/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1_oSwxT18YmiRhcQTT6jPFRg3LlgQd9SlI9cnSfMgFro/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/19h_FBsM23Hy2lYyXhzvWZIdPPIuxcNeh4fyAASwpLRM/edit#gid=311031188'
]

regions = {'midwest': ['Nebraska', 'Iowa', 'Kentucky'],
                    'mountains': ['Colorado', 'Idaho', 'Montana', 'New Mexico', 'South Dakota', 'Wyoming'],
                    'prairie_mrv': ['Oklahoma', 'Louisiana', 'Kansas', 'Arkansas', 'Mississippi', 'Missouri'],
                    'northeast': ['Delaware', 'Maryland', 'New Hampshire', 'New York', 'Rhode Island', 'Washington', 'New Jersey'],
                    'southeast': ['Alabama', 'Florida', 'North Carolina', 'South Carolina', 'West Virginia'],
                    'w_pacific' : ['Arizona', 'Nevada', 'Utah', 'Oregon', 'Alaska', 'Hawaii']
                    }

all_regions = []
for i in url_lst:
    sheet_url = i
    url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df = pd.read_csv(url_1)

    # insert date after the day you want to stop checking for trend breaks
    all_regions.append(trend_break(df, '5/16/2022'))

In [51]:
midwest = {}
mountains = {}
prairie_mrv = {}
northeast = {}
southeast = {}
w_pacific = {}

for sheet in all_regions:
    for date in sheet:
        for state in sheet[date]:
            for region in regions:
                if state in regions[region]:
                    curr = eval(region)
                    if date not in curr.keys():
                        curr[date] = [state]
                    else:
                        curr[date] += [state]

In [71]:
midwest = dict(OrderedDict(sorted(midwest.items(), key=lambda t: t[0])))
midwest

{'3/30/2022': ['Nebraska', 'Iowa'],
 '3/31/2022': ['Nebraska'],
 '4/11/2022': ['Kentucky'],
 '4/13/2022': ['Iowa'],
 '4/17/2022': ['Kentucky'],
 '4/2/2022': ['Nebraska'],
 '4/20/2022': ['Iowa'],
 '4/25/2022': ['Nebraska', 'Kentucky'],
 '4/26/2022': ['Nebraska'],
 '4/27/2022': ['Iowa'],
 '4/29/2022': ['Kentucky'],
 '4/4/2022': ['Kentucky'],
 '4/5/2022': ['Nebraska'],
 '4/6/2022': ['Iowa'],
 '4/8/2022': ['Nebraska'],
 '5/10/2022': ['Nebraska'],
 '5/11/2022': ['Nebraska', 'Kentucky'],
 '5/12/2022': ['Nebraska'],
 '5/2/2022': ['Nebraska', 'Kentucky'],
 '5/3/2022': ['Nebraska'],
 '5/4/2022': ['Iowa']}

In [70]:
mountains = dict(OrderedDict(sorted(mountains.items(), key=lambda t: t[0])))
mountains

{'3/29/2022': ['Colorado', 'Wyoming', 'Idaho'],
 '3/30/2022': ['Montana', 'Colorado', 'South Dakota', 'Idaho'],
 '3/31/2022': ['Colorado', 'Idaho'],
 '4/1/2022': ['Montana', 'Colorado', 'Wyoming', 'Idaho'],
 '4/11/2022': ['Montana', 'Colorado', 'Idaho'],
 '4/12/2022': ['New Mexico', 'Montana', 'Wyoming', 'Idaho'],
 '4/13/2022': ['New Mexico', 'Montana', 'South Dakota'],
 '4/14/2022': ['New Mexico', 'Montana', 'Colorado', 'Idaho'],
 '4/15/2022': ['Montana', 'Colorado', 'Idaho'],
 '4/19/2022': ['New Mexico', 'Montana', 'Wyoming'],
 '4/20/2022': ['Montana', 'South Dakota', 'Colorado', 'Idaho'],
 '4/21/2022': ['Montana'],
 '4/22/2022': ['Montana', 'Wyoming', 'Colorado', 'Idaho'],
 '4/25/2022': ['New Mexico', 'Montana', 'Colorado', 'Idaho'],
 '4/26/2022': ['Colorado', 'Wyoming'],
 '4/27/2022': ['Montana', 'Colorado', 'Idaho'],
 '4/28/2022': ['New Mexico', 'Montana'],
 '4/29/2022': ['Montana', 'Idaho'],
 '4/4/2022': ['Colorado'],
 '4/5/2022': ['Montana', 'Colorado'],
 '4/6/2022': ['Colorado'

In [55]:
prairie_mrv

{'3/29/2022': ['Missouri', 'Arkansas'],
 '3/30/2022': ['Louisiana'],
 '3/31/2022': ['Arkansas'],
 '4/1/2022': ['Kansas', 'Missouri'],
 '4/2/2022': ['Missouri'],
 '4/4/2022': ['Missouri'],
 '4/5/2022': ['Kansas'],
 '4/6/2022': ['Louisiana'],
 '4/8/2022': ['Kansas', 'Missouri'],
 '4/9/2022': ['Arkansas', 'Missouri'],
 '4/10/2022': ['Arkansas'],
 '4/12/2022': ['Louisiana'],
 '4/15/2022': ['Kansas'],
 '4/16/2022': ['Arkansas'],
 '4/19/2022': ['Arkansas'],
 '4/21/2022': ['Louisiana', 'Oklahoma', 'Arkansas'],
 '4/22/2022': ['Kansas', 'Missouri', 'Arkansas'],
 '4/23/2022': ['Missouri'],
 '4/24/2022': ['Arkansas'],
 '4/25/2022': ['Arkansas', 'Missouri'],
 '4/26/2022': ['Arkansas'],
 '4/27/2022': ['Louisiana'],
 '4/28/2022': ['Louisiana', 'Oklahoma', 'Missouri'],
 '4/29/2022': ['Oklahoma', 'Kansas'],
 '4/30/2022': ['Arkansas'],
 '5/4/2022': ['Louisiana'],
 '5/5/2022': ['Louisiana', 'Oklahoma', 'Arkansas'],
 '5/6/2022': ['Oklahoma'],
 '5/7/2022': ['Arkansas'],
 '5/10/2022': ['Louisiana', 'Arkans

In [65]:
northeast = dict(OrderedDict(sorted(northeast.items(), key=lambda t: t[0])))
northeast

{'3/29/2022': ['Maryland'],
 '3/30/2022': ['New York', 'Delaware', 'Rhode Island'],
 '3/31/2022': ['New York', 'New Hampshire', 'Maryland'],
 '4/1/2022': ['New York', 'New Jersey', 'Maryland'],
 '4/10/2022': ['New Jersey'],
 '4/11/2022': ['New Hampshire'],
 '4/12/2022': ['New Hampshire', 'New Jersey'],
 '4/13/2022': ['Rhode Island', 'Maryland', 'New Jersey'],
 '4/14/2022': ['Maryland', 'New Hampshire'],
 '4/15/2022': ['New Jersey', 'Maryland'],
 '4/16/2022': ['New Jersey'],
 '4/17/2022': ['Delaware', 'New Jersey'],
 '4/19/2022': ['New Hampshire', 'New Jersey'],
 '4/2/2022': ['New York', 'New Jersey'],
 '4/20/2022': ['Rhode Island',
  'New York',
  'New Hampshire',
  'Maryland',
  'New Jersey'],
 '4/21/2022': ['New York', 'Delaware'],
 '4/22/2022': ['Maryland'],
 '4/23/2022': ['New York',
  'New Jersey',
  'New Hampshire',
  'Maryland',
  'Rhode Island'],
 '4/25/2022': ['New York', 'Maryland', 'New Jersey'],
 '4/26/2022': ['New York'],
 '4/27/2022': ['New York', 'Maryland', 'Rhode Islan

In [66]:
southeast = dict(OrderedDict(sorted(southeast.items(), key=lambda t: t[0])))
southeast

{'3/29/2022': ['West Virginia'],
 '3/30/2022': ['Alabama'],
 '3/31/2022': ['West Virginia', 'Alabama'],
 '4/11/2022': ['West Virginia'],
 '4/12/2022': ['Alabama', 'South Carolina'],
 '4/13/2022': ['North Carolina', 'West Virginia', 'Alabama'],
 '4/14/2022': ['Alabama', 'West Virginia'],
 '4/15/2022': ['Alabama'],
 '4/19/2022': ['Alabama'],
 '4/20/2022': ['North Carolina', 'Alabama'],
 '4/21/2022': ['Alabama', 'West Virginia'],
 '4/25/2022': ['Alabama'],
 '4/27/2022': ['North Carolina', 'Alabama'],
 '4/28/2022': ['Alabama', 'West Virginia'],
 '4/29/2022': ['Alabama'],
 '4/4/2022': ['North Carolina', 'West Virginia', 'Alabama'],
 '4/5/2022': ['North Carolina', 'West Virginia', 'South Carolina'],
 '4/6/2022': ['North Carolina', 'West Virginia', 'Alabama'],
 '4/7/2022': ['Alabama'],
 '4/8/2022': ['Alabama'],
 '5/10/2022': ['South Carolina'],
 '5/11/2022': ['Alabama', 'North Carolina'],
 '5/12/2022': ['Alabama'],
 '5/13/2022': ['Alabama'],
 '5/2/2022': ['West Virginia'],
 '5/3/2022': ['West

In [67]:
w_pacific = dict(OrderedDict(sorted(w_pacific.items(), key=lambda t: t[0])))
w_pacific

{'3/29/2022': ['Utah', 'Oregon'],
 '3/30/2022': ['Hawaii', 'Arizona', 'Oregon'],
 '3/31/2022': ['Oregon'],
 '4/1/2022': ['Nevada', 'Oregon'],
 '4/11/2022': ['Oregon'],
 '4/12/2022': ['Oregon'],
 '4/13/2022': ['Arizona', 'Nevada'],
 '4/15/2022': ['Utah'],
 '4/16/2022': ['Oregon'],
 '4/19/2022': ['Oregon'],
 '4/2/2022': ['Nevada'],
 '4/20/2022': ['Alaska', 'Hawaii', 'Nevada', 'Utah', 'Arizona'],
 '4/21/2022': ['Nevada'],
 '4/22/2022': ['Utah', 'Oregon'],
 '4/24/2022': ['Hawaii'],
 '4/27/2022': ['Nevada', 'Arizona', 'Oregon'],
 '4/29/2022': ['Alaska'],
 '4/4/2022': ['Nevada'],
 '4/6/2022': ['Nevada', 'Oregon'],
 '4/7/2022': ['Utah', 'Oregon'],
 '4/9/2022': ['Nevada'],
 '5/11/2022': ['Nevada'],
 '5/12/2022': ['Utah'],
 '5/15/2022': ['Utah'],
 '5/2/2022': ['Nevada'],
 '5/3/2022': ['Utah', 'Nevada'],
 '5/4/2022': ['Alaska', 'Nevada', 'Utah'],
 '5/5/2022': ['Oregon']}